In [25]:
from gensim.models import Word2Vec, KeyedVectors
import matplotlib.pyplot as plt
from pyvi import ViTokenizer
import pandas as pd
import numpy as np

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# %cd /content/drive/My Drive/Colab Notebooks/Text_Generator

/content/drive/My Drive/Colab Notebooks/Text_Generator


In [ ]:
# %ls -la

In [2]:
news = pd.read_csv('./data/vnexpress/csv/vnexpress.csv').drop(columns='Unnamed: 0', axis=1)
news.head()

,article_id,content,topic,sub-topic,title,description
0,0,Người mẹ 42 tuổi ở Đà Nẵng vội vã tra hỏi và v...,doi-song,to-am,Cha mẹ học về giáo dục giới tính để dạy con,Nghe con trai 14 tuổi nói đã làm cho chị hàng ...
1,1,"Vợ chồng người phụ nữ ở Mỹ Đức, Hà Nội luôn tự...",doi-song,to-am,Nỗi lòng của phụ huynh có con mang thai tuổi đ...,Khi con gái đang là sinh viên năm nhất một trư...
2,2,"Kati Morton, nhà trị liệu hôn nhân và gia đình...",doi-song,to-am,Giải mã 'Hội chứng con gái đầu lòng',"""Hội chứng con gái đầu lòng"" cho rằng con gái ..."
3,3,"Anh Thắng, chủ tịch Hội làm vườn tỉnh Nghệ An ...",doi-song,to-am,Vườn sân thượng như khu rừng giải nhiệt mùa hè,Nghệ AnMuốn hạ nhiệt cho ngôi nhà khi nắng nón...
4,4,"Vừa kết thúc môn tiếng Anh, Phương bị sốt cao,...",doi-song,to-am,Nghị lực của cô bé ung thư,"Hà NamMột tuần trước, cô bé 14 tuổi từ Bệnh vi..."


In [3]:
news['tag'] = news['content'] + news['topic'] + news['sub-topic'] + news['title'] + news['description']
news = news[['tag']]

In [4]:
news.head()

,tag
0,Người mẹ 42 tuổi ở Đà Nẵng vội vã tra hỏi và v...
1,"Vợ chồng người phụ nữ ở Mỹ Đức, Hà Nội luôn tự..."
2,"Kati Morton, nhà trị liệu hôn nhân và gia đình..."
3,"Anh Thắng, chủ tịch Hội làm vườn tỉnh Nghệ An ..."
4,"Vừa kết thúc môn tiếng Anh, Phương bị sốt cao,..."


In [7]:
news.isna().sum()

tag    0
dtype: int64

In [6]:
news.dropna(axis=0, inplace=True)

In [8]:
news['tag'] = news['tag'].apply(lambda x: x.lower())
news['tag'] = news['tag'].apply(ViTokenizer.tokenize)

In [9]:
news

,tag
0,người mẹ 42 tuổi ở đà_nẵng vội_vã tra_hỏi và v...
1,"vợ_chồng người phụ_nữ ở mỹ đức , hà_nội luôn t..."
2,"kati morton , nhà trị_liệu hôn_nhân và gia_đìn..."
3,"anh thắng , chủ_tịch hội làm_vườn tỉnh nghệ_an..."
4,"vừa kết_thúc môn tiếng anh , phương bị sốt cao..."
...,...
1342,"từ ngày 4 / 4 , xe_khách trên 30 chỗ , xe giườ..."
1343,dự_án đang được ubnd tỉnh xem_xét trình hđnd t...
1344,"ngày 1 / 4 , ban an_toàn giao_thông tỉnh thừa ..."
1345,nhằm đồng_bộ các dự_án cao_tốc bắc_nam 4 làn x...


In [21]:
sequences = news['tag'].to_list()
input_gensim = []

for sen in sequences:
  input_gensim.append(sen.split())

1256

In [ ]:
model = Word2Vec(input_gensim, vector_size=50, window=5, min_count=0, workers=4, sg=1)
model.wv.save("./model/word.model")

In [28]:
model = KeyedVectors.load("./model/word.model")

In [69]:
question = 'trong giây phút đoàn tụ , gia đình ôm chặt lấy nhau khóc nức nở'

def embedding(question, input_gensim):
  question_tokens = ViTokenizer.tokenize(question)
  print('After tonkenized:', question_tokens)
  
  # Word embeddings
  question_embeddings = [model[word] for word in question_tokens.split() if word in model.index_to_key]

  # Post embeddings
  post_check = input_gensim[:20]

  post_embeddings = []
  for post in post_check:
    post_embedding = [model[word] for word in post if word in model.index_to_key]
    post_embeddings.append(post_embedding)

  return question_embeddings, post_embeddings

question_embeddings, post_embeddings = embedding(question, input_gensim)

After tonkenized: trong giây_phút đoàn_tụ , gia_đình ôm chặt lấy nhau khóc nức_nở


In [70]:
print(len(question_embeddings))
question_embeddings[0]

11


array([ 0.02640989, -0.29494327, -0.09446201,  0.16745777, -0.20994249,
       -0.59924686,  1.0844842 , -0.1060454 , -0.39822787, -0.17184252,
        0.335344  , -0.40530542,  0.13094893, -0.29672518, -0.16974089,
        0.48377067,  0.03868706, -0.18952471, -0.3054984 , -0.46489137,
        0.1384956 ,  0.7928218 , -0.03043196, -0.34931725,  0.09405179,
        0.6689448 , -0.6228975 ,  0.23988815,  0.03245716, -0.11234264,
        0.33866194, -0.12129344, -0.05845054, -0.29715687,  0.234383  ,
        0.00943883, -0.01900785,  0.16458702, -0.63906294, -0.11707322,
        0.01822926,  0.13106883, -0.6643712 , -0.23594822,  0.35835832,
        0.2264553 , -0.3534701 , -0.42675343,  0.4369625 ,  0.09607002],
      dtype=float32)

In [71]:
print(len(post_embeddings))
post_embeddings[0][0]

20


array([ 0.31542802, -0.47440916, -0.52656746, -0.2929978 , -0.49965146,
       -0.5454484 ,  0.5848612 ,  0.9104822 ,  0.19063218, -0.2737768 ,
        0.2152797 , -0.2697827 ,  0.368683  ,  0.08309438,  0.10788494,
       -0.04328385, -0.13117217,  0.23456934, -0.44848463, -1.0196686 ,
        0.6732069 ,  0.3084358 ,  0.19239073,  0.3811673 ,  0.05862541,
       -0.03174697, -0.35160512,  0.5376984 , -0.2615627 ,  0.05475898,
        0.05937112,  0.506843  ,  0.3482966 , -0.0884548 , -0.27384698,
        0.02474154,  0.4895178 ,  0.3811738 , -0.54975593, -0.5344599 ,
       -0.1110735 , -0.03929105, -0.5265459 , -0.27379063,  0.7492893 ,
        0.29412547,  0.2087791 , -0.22899292,  0.2949489 , -0.2510092 ],
      dtype=float32)

In [34]:
from sklearn.metrics.pairwise import cosine_similarity

In [86]:
# Calculate sentence embeddings by averaging word embeddings
def mean_vector_embedding(embeddings):
  if len(embeddings) == 0:
      return np.zeros(model.vector_size)
  return np.mean(embeddings, axis=0)

mean_sentence_embedding = mean_vector_embedding(question_embeddings)
print('The mean of an embedding vector:', mean_sentence_embedding)

def mean_embedded_posts(post_embeddings):
  mean_post_embedding = []
  for post_embedding in post_embeddings:
      mean_post_embedding.append(mean_vector_embedding(post_embedding))

  return mean_post_embedding

mean_post_embedding = mean_embedded_posts(post_embeddings)
print('The mean of embedding posts:', len(mean_post_embedding))

# Calculate similarity (cosine similarity)
similarity_score = cosine_similarity([mean_sentence_embedding], mean_post_embedding)

# Print the similarity score
print(f"Similarity Score: {similarity_score}")

The mean of an embedding vector: [-0.13599294  0.01352213 -0.04441348  0.15260603 -0.17083478 -0.45150012
  0.340879    0.46188134 -0.31045336 -0.03323688  0.0724034  -0.21073675
  0.02078545  0.06246011 -0.31023932  0.17357683 -0.06676442 -0.19114985
 -0.19788261 -0.46371222  0.04825732  0.3209744   0.15918878 -0.11662194
 -0.03570771  0.28439933 -0.14423797  0.14450473 -0.15446721  0.13133496
  0.16882442  0.09717653 -0.09915912  0.07364793 -0.00553187  0.05989678
  0.15053314  0.17351943 -0.18611936 -0.16123188 -0.01329542 -0.01260709
 -0.31070143  0.00696368  0.44915962  0.1769488   0.04660099 -0.21090029
  0.19013019  0.08098011]
The mean of embedding posts: 20
Similarity Score: [[0.97503674 0.98028713 0.97952247 0.96645373 0.9810803  0.9828994
  0.963122   0.9807412  0.9803909  0.9804094  0.97121227 0.97961265
  0.9604981  0.9775699  0.95835364 0.9792127  0.98388404 0.9670023
  0.9773519  0.9690767 ]]


In [78]:
# Convert the list of lists into a numpy array
np_similarity_score = np.array(similarity_score)
print('np_question_embedding', np_similarity_score)
# Sort the array in ascending order
sorted_indices = np.argsort(np_similarity_score[0])[::-1]

# Get the top 5 indices
top_5_indices = sorted_indices[:5]

# Print the sorted indices and the top 5 indices
print("Sorted Indices:", sorted_indices)
print("Top 5 Indices:", top_5_indices)

np_question_embedding [[0.97503674 0.98028713 0.97952247 0.96645373 0.9810803  0.9828994
  0.963122   0.9807412  0.9803909  0.9804094  0.97121227 0.97961265
  0.9604981  0.9775699  0.95835364 0.9792127  0.98388404 0.9670023
  0.9773519  0.9690767 ]]
Sorted Indices: [16  5  4  7  9  8  1 11  2 15 13 18  0 10 19 17  3  6 12 14]
Top 5 Indices: [16  5  4  7  9]


In [79]:
for indice in top_5_indices:
    # Remove duplicate spaces
    cleaned_sentence = ' '.join(sequences[indice][:200].split())
    print("Bài viết thứ " + str(indice) + ":", cleaned_sentence)

Bài viết thứ 16: cũng bởi tình_yêu này , cô gái 23 tuổi muốn bản_thân trở_nên xinh_đẹp hơn trong mắt bạn trai . một năm trước , hường đăng_ký tham_gia chương_trình " hành_trình lột_xác " của một bệnh_viện tại hà_nội đ
Bài viết thứ 5: hoàng thị thoa , 33 tuổi , quê nghệ_an kết_hôn ở tuổi 26 , nhưng khi mang thai tháng thứ tư thì phải đình_chỉ thai kỳ . trong nỗi đau mất con , cô cũng phát_hiện chồng phản_bội . công_ty của hai vợ_ch
Bài viết thứ 4: vừa kết_thúc môn tiếng anh , phương bị sốt cao , lập_tức được mẹ đưa lại viện cấp_cứu . trước khi đẩy vào phòng làm xét_nghiệm , cô bé gắng nhoài người dặn_dò : " hết sốt , mẹ lại xin bác_sĩ cho con v
Bài viết thứ 7: muốn vậy , cha_mẹ phải dạy con cách_trở_nên mạnh_mẽ . nghiên_cứu cho thấy những người có tinh_thần dẻo_dai , bền_bỉ thể_hiện một loạt đặc_điểm và phẩm_chất như tự_tin , kiên_cường , kiên_trì , để giúp
Bài viết thứ 9: trần , 49 tuổi , là người điều_hành " chung_cư cầu con " tại thượng_hải . cô cũng làm ivf ( thụ_tinh trong ống_nghi